## Objective:
Create relationship between the Cabin study data and segments and/or water bodies.

In [1]:
import geopandas as gpd
import pandas as pd
import fiona
from ipyfilechooser import FileChooser
from shapely.geometry import Polygon, LineString, Point
from geopandas import GeoSeries

## Steps:
- Convert lon and lot from Cabin data to Points
- Use the two functions in `Barrier-Waterbody-segments-connection` file
- Find the closest segment and water body to each study point
- Export the results as CSv for Neo4j